In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import gender_guesser.detector as gender
from functools import reduce

# Aggregate all variables

In [2]:
## dependent variables and control variables
info = pd.read_csv('data2.csv', sep=',')
## human portrait
face_df = pd.read_csv('scores/seller_pictures_info.csv', sep='\t')
## facial attractiveness
beauty_df = pd.read_csv('scores/seller_beauty.csv', sep='\t')
## image quality
raw_qua_df = pd.read_csv('scores/seller_pictures_quality.csv', sep='\t')
qua = - raw_qua_df.quality.values
qua = (qua - np.nanmin(qua)) / (np.nanmax(qua) - np.nanmin(qua))
qua_df = raw_qua_df
qua_df['quality'] = qua

In [3]:
## smiling & intensity
emo_df = pd.read_csv('scores/seller_emotion.csv', sep='\t')
## asymmetry
asym_df = pd.read_csv('scores/seller_asymmetry.csv', sep='\t')
## gender estimation
d = gender.Detector()
genders = []
for sid in tqdm(set(info.seller_ID.values.tolist())):
    name = info.loc[info.seller_ID == sid].seller_name.values[0]
    genders.append(d.get_gender(name.split(' ')[0]))
female_ids = [i for i in range(len(genders)) if genders[i] == 'female' or genders[i] == 'mostly_female']
gender_ind = np.zeros(len(genders))
gender_ind[female_ids] = 1
gender_df = pd.DataFrame({'seller_ID': list(set(info.seller_ID.values.tolist())), 'female': gender_ind.astype(int)})
## aggregate all variables
dfs = [info, gender_df, qua_df, face_df, beauty_df, emo_df[['seller_ID', 'smile', 'Intensity']], asym_df[['seller_ID', 'asymmetry']]]
var_df = reduce(lambda left, right: pd.merge(left, right, on='seller_ID', how='outer'), dfs)
var_df['offered'] = ((var_df.offer_made > 0) | (var_df.product_sold == 1)).astype(int)
var_df = var_df.sort_values('ID').reset_index(drop=True)
var_df.to_csv('scores/variables.csv', sep='\t', index=False)

100%|██████████| 11165/11165 [00:18<00:00, 588.98it/s]


# Summary statistics

In [4]:
## profile picture
print(f'Presence of picture: {len(var_df.loc[var_df.seller_pictures.isna()==False])}, {np.mean(var_df.seller_pictures.isna()==False):.3f}, {np.std(var_df.seller_pictures.isna()==False):.3f}')
print(f'Human portrait: {len(var_df.loc[var_df.num_faces > 0])}, {np.mean((var_df.num_faces > 0).values):.3f}, {np.std((var_df.num_faces > 0).values):.3f}')
print(f'Presence of smiling: {len(var_df.loc[var_df.smile.isna()==False])}, {np.mean(var_df.loc[var_df.smile.isna()==False].smile.values):.3f}, {np.std(var_df.loc[var_df.smile.isna()==False].smile.values):.3f}')
print(f'Smiling intensity: {len(var_df.loc[var_df.Intensity.isna()==False])}, {np.mean(var_df.loc[var_df.Intensity.isna()==False].Intensity.values):.3f}, {np.std(var_df.loc[var_df.Intensity.isna()==False].Intensity.values):.3f},\
      {np.min(var_df.loc[var_df.Intensity.isna()==False].Intensity.values):.3f}, {np.max(var_df.loc[var_df.Intensity.isna()==False].Intensity.values):.3f}')
print(f'Smiling asymmetry: {len(var_df.loc[var_df.asymmetry.isna()==False])}, {np.mean(var_df.loc[var_df.asymmetry.isna()==False].asymmetry.values):.3f}, {np.std(var_df.loc[var_df.asymmetry.isna()==False].asymmetry.values):.3f},\
      {np.min(var_df.loc[var_df.asymmetry.isna()==False].asymmetry.values):.3f}, {np.max(var_df.loc[var_df.asymmetry.isna()==False].asymmetry.values):.3f}')
print(f'Photographic quality: {len(var_df.loc[var_df.seller_pictures.isna()==False])}, {np.mean(var_df.loc[var_df.quality.isna()==False].quality.values):.3f}, {np.std(var_df.loc[var_df.quality.isna()==False].quality.values):.3f},\
      {np.min(var_df.loc[var_df.quality.isna()==False].quality.values):.3f}, {np.max(var_df.loc[var_df.quality.isna()==False].quality.values):.3f}')
print(f'Facial attractiveness: {len(var_df.loc[var_df.beauty.isna()==False])}, {np.mean(var_df.loc[var_df.beauty.isna()==False].beauty.values):.3f}, {np.std(var_df.loc[var_df.beauty.isna()==False].beauty.values):.3f},\
      {np.min(var_df.loc[var_df.beauty.isna()==False].beauty.values):.3f}, {np.max(var_df.loc[var_df.beauty.isna()==False].beauty.values):.3f}')
print(f'Face proximity: {len(var_df.loc[var_df.face_proximity.isna()==False])}, {np.mean(var_df.loc[var_df.face_proximity.isna()==False].face_proximity.values):.3f}, {np.std(var_df.loc[var_df.face_proximity.isna()==False].face_proximity.values):.3f},\
      {np.min(var_df.loc[var_df.face_proximity.isna()==False].face_proximity.values):.3f}, {np.max(var_df.loc[var_df.face_proximity.isna()==False].face_proximity.values):.3f}')


Presence of picture: 22460, 0.853, 0.354
Human portrait: 10073, 0.382, 0.486
Presence of smiling: 8333, 0.461, 0.498
Smiling intensity: 8333, 0.424, 0.259,      0.008, 1.273
Smiling asymmetry: 8333, 0.561, 0.488,      0.000, 1.414
Photographic quality: 22460, 0.693, 0.093,      0.000, 1.000
Facial attractiveness: 8333, 3.216, 0.581,      1.487, 4.942
Face proximity: 8333, 0.210, 0.162,      0.000, 1.000


In [5]:
## seller characteristics
print(f'Female: {len(var_df.loc[var_df.female.isna()==False])}, {np.mean(var_df.loc[var_df.female.isna()==False].female.values):.3f}, {np.std(var_df.loc[var_df.female.isna()==False].female.values):.3f}')
print(f'Trust level: {len(var_df.loc[var_df.seller_level.isna()==False])}, {np.mean(var_df.loc[var_df.seller_level.isna()==False].seller_level.values):.3f}, {np.std(var_df.loc[var_df.seller_level.isna()==False].seller_level.values):.3f},\
      {np.min(var_df.loc[var_df.seller_level.isna()==False].seller_level.values):.3f}, {np.max(var_df.loc[var_df.seller_level.isna()==False].seller_level.values):.3f}')
print(f'Verified email: {len(var_df.loc[var_df.seller_verification_email.isna()==False])}, {np.mean(var_df.loc[var_df.seller_verification_email.isna()==False].seller_verification_email.values):.3f}, {np.std(var_df.loc[var_df.seller_verification_email.isna()==False].seller_verification_email.values):.3f},\
      {np.min(var_df.loc[var_df.seller_verification_email.isna()==False].seller_verification_email.values):.3f}, {np.max(var_df.loc[var_df.seller_verification_email.isna()==False].seller_verification_email.values):.3f}')
print(f'Verified phone number: {len(var_df.loc[var_df.seller_verification_phone.isna()==False])}, {np.mean(var_df.loc[var_df.seller_verification_phone.isna()==False].seller_verification_phone.values):.3f}, {np.std(var_df.loc[var_df.seller_verification_phone.isna()==False].seller_verification_phone.values):.3f},\
      {np.min(var_df.loc[var_df.seller_verification_phone.isna()==False].seller_verification_phone.values):.3f}, {np.max(var_df.loc[var_df.seller_verification_phone.isna()==False].seller_verification_phone.values):.3f}')
print(f'Linked facebook account: {len(var_df.loc[var_df.seller_verification_fb.isna()==False])}, {np.mean(var_df.loc[var_df.seller_verification_fb.isna()==False].seller_verification_fb.values):.3f}, {np.std(var_df.loc[var_df.seller_verification_fb.isna()==False].seller_verification_fb.values):.3f},\
      {np.min(var_df.loc[var_df.seller_verification_fb.isna()==False].seller_verification_fb.values):.3f}, {np.max(var_df.loc[var_df.seller_verification_fb.isna()==False].seller_verification_fb.values):.3f}')

Female: 26341, 0.385, 0.487
Trust level: 26341, 3.293, 2.520,      0.000, 11.000
Verified email: 26341, 0.751, 0.433,      0.000, 1.000
Verified phone number: 26341, 0.950, 0.218,      0.000, 1.000
Linked facebook account: 26341, 0.384, 0.486,      0.000, 1.000


In [6]:
## product characteristics
print(f'Number of product photos: {len(var_df.loc[var_df.product_images.isna()==False])}, {np.mean(var_df.loc[var_df.product_images.isna()==False].product_images.values):.3f}, {np.std(var_df.loc[var_df.product_images.isna()==False].product_images.values):.3f},\
      {np.min(var_df.loc[var_df.product_images.isna()==False].product_images.values):.3f}, {np.max(var_df.loc[var_df.product_images.isna()==False].product_images.values):.3f}')
print(f'Log length of listing description: {len(var_df.loc[var_df.product_desc_length.isna()==False])}, {np.nanmean(np.log(var_df.loc[var_df.product_desc_length.isna()==False].product_desc_length.values)):.3f}, {np.nanstd(np.log(var_df.loc[var_df.product_desc_length.isna()==False].product_desc_length.values)):.3f},\
      {np.nanmin(np.log(var_df.loc[var_df.product_desc_length.isna()==False].product_desc_length.values)):.3f}, {np.nanmax(np.log(var_df.loc[var_df.product_desc_length.isna()==False].product_desc_length.values)):.3f}')
print(f'Price of the product: {len(var_df.loc[var_df.product_price.isna()==False])}, {np.mean(var_df.loc[var_df.product_price.isna()==False].product_price.values):.3f}, {np.std(var_df.loc[var_df.product_price.isna()==False].product_price.values):.3f},\
      {np.min(var_df.loc[var_df.product_price.isna()==False].product_price.values):.3f}, {np.max(var_df.loc[var_df.product_price.isna()==False].product_price.values):.3f}')
print(f'Offer made by buyers: {len(var_df.loc[var_df.offered.isna()==False])}, {np.mean(var_df.loc[var_df.offered.isna()==False].offered.values):.3f}, {np.std(var_df.loc[var_df.offered.isna()==False].offered.values):.3f},\
      {np.min(var_df.loc[var_df.offered.isna()==False].offered.values):.3f}, {np.max(var_df.loc[var_df.offered.isna()==False].offered.values):.3f}')

Number of product photos: 26341, 2.994, 2.235,      0.000, 12.000
Log length of listing description: 26341, 2.273, 1.347,      0.000, 7.056
Price of the product: 26341, 7023.512, 650595.697,      0.000, 85886555.000
Offer made by buyers: 26341, 0.473, 0.499,      0.000, 1.000
